<a href="https://colab.research.google.com/github/andresit02/Web---Scraping/blob/main/Actividad_extracurricular_12_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Web scraping utilizando dos bibliotecas populares: BeautifulSoup y Scrapy**

### **BEATIFULSHOP**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_quotes():
    """
    Función para extraer y mostrar frases de manera estructurada
    """
    try:
        # Realizar la petición
        print("Conectando al sitio web...\n")
        page = requests.get('https://parascrapear.com/')
        soup = BeautifulSoup(page.text, 'html.parser')

        # Lista para almacenar los resultados
        quotes_list = []

        # Encontrar todas las citas
        blockquote_items = soup.find_all('blockquote')

        print("=== FRASES ENCONTRADAS ===")
        print("------------------------")

        for i, blockquote in enumerate(blockquote_items, 1):
            # Extraer información
            autor = blockquote.find(class_='author').text.strip()
            categoria = blockquote.find(class_='cat').text.strip()
            frase = blockquote.find('q').text.strip()

            # Almacenar en diccionario
            quote_dict = {
                'autor': autor,
                'categoria': categoria,
                'frase': frase
            }
            quotes_list.append(quote_dict)

            # Mostrar de manera formateada
            print(f"\nCita #{i}")
            print("------------------------")
            print(f"Autor: {autor}")
            print(f"Categoría: {categoria}")
            print(f"Frase: \"{frase}\"")
            print("------------------------")

        # Crear DataFrame
        df = pd.DataFrame(quotes_list)

        # Guardar en CSV con timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'frases_{timestamp}.csv'
        df.to_csv(filename, index=False, encoding='utf-8')

        print(f"\nTotal de frases encontradas: {len(quotes_list)}")
        print(f"Los resultados se han guardado en: {filename}")

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error de conexión: {str(e)}")
        return None
    except Exception as e:
        print(f"Error inesperado: {str(e)}")
        return None

def mostrar_estadisticas(df):
    """
    Muestra estadísticas sobre las frases recolectadas
    """
    if df is not None and not df.empty:
        print("\n=== ESTADÍSTICAS ===")
        print(f"Número total de autores únicos: {df['autor'].nunique()}")
        print(f"Número total de categorías: {df['categoria'].nunique()}")
        print("\nTop 3 categorías más comunes:")
        print(df['categoria'].value_counts().head(3))

if __name__ == "__main__":
    print("Iniciando web scraping de frases...\n")
    df = scrape_quotes()
    if df is not None:
        mostrar_estadisticas(df)

Iniciando web scraping de frases...

Conectando al sitio web...

=== FRASES ENCONTRADAS ===
------------------------

Cita #1
------------------------
Autor: Blaise Pascal
Categoría: Sociedad
Frase: "Prefiero equivocarme creyendo en un Dios que no existe, que equivocarme no creyendo en un Dios que existe. Porque si después no hay nada, evidentemente nunca lo sabré, cuando me hunda en la nada eterna; pero si hay algo, si hay Alguien, tendré que dar cuenta de mi actitud de rechazo."
------------------------

Cita #2
------------------------
Autor: Jean Jacques Rousseau
Categoría: Sociedad
Frase: "El hombre ha nacido libre y por doquiera se encuentra sujeto con cadenas."
------------------------

Cita #3
------------------------
Autor: Blaise Pascal
Categoría: Naturaleza
Frase: "Nuestra naturaleza está en movimiento. El reposo absoluto es la muerte."
------------------------

Cita #4
------------------------
Autor: Mahatma Gandhi
Categoría: Defectos
Frase: "Quisiera sufrir todas las humil

## **SCRAPY**

In [ ]:
import requests # pip install requests
from lxml import html # pip install lxml

# USER AGENT PARA PROTEGERNOS DE BANEOS
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

# URL SEMILLA
url = 'https://www.wikipedia.org/'

# REQUERIMIENTO AL SERVIDOR
respuesta = requests.get(url, headers=headers)
respuesta.encoding = 'utf-8' # Codificar correctamente caracteres extranos

# PARSEO DEL ARBOL HTML QUE RECIBO COMO RESPUESTA CON LXML
parser = html.fromstring(respuesta.content) # Uso .content para poder codificar los caracteres raros

# EXTRACCION DE IDIOMA INGLES
ingles = parser.get_element_by_id("js-link-box-en")
print (ingles.text_content())

# EXTRACCION SOLO DEL TEXTO QUE DICE INGLES
ingles = parser.xpath("//a[@id='js-link-box-en']/strong/text()")
print(ingles[0])

# EXTRACCION DE TODOS LOS IDIOMAS POR CLASE
idiomas = parser.find_class('central-featured-lang')
for idioma in idiomas:
  print(idioma.text_content())

# EXTRACCION DE TODOS LOS IDIOMAS POR XPATH
idiomas = parser.xpath("//div[contains(@class,'central-featured-lang')]//strong/text()")
for idioma in idiomas:
  print(idioma)


English
6,935,000+ articles

English


English
6,935,000+ articles




日本語
1,443,000+ 記事




Русский
2 017 000+ статей




Deutsch
2.975.000+ Artikel




Français
2 657 000+ articles




Español
2.000.000+ artículos




Italiano
1.898.000+ voci




中文
1,457,000+ 条目 / 條目




فارسی
۱٬۰۲۴٬۰۰۰+ مقاله




Polski
1 642 000+ haseł


English
日本語
Русский
Deutsch
Français
Español
Italiano
中文
Polski


**Web scraping de un sitio web de mi elección**

###**MARATHON SPORTS**

In [ ]:
import scrapy
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class MarathonSpider(scrapy.Spider):
    name = "marathon_spider"
    start_urls = ["https://www.marathon.store/ec/"]

    def __init__(self):
        self.driver = webdriver.Chrome()

    def parse(self, response):
        self.driver.get(response.url)


        WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product-card")))



        html = self.driver.page_source
        response = scrapy.http.HtmlResponse(url=self.driver.current_url, body=html.encode('utf-8'), encoding='utf-8')



        for product in response.css(".product-card"):
            yield {
                'title': product.css("YOUR_CSS_SELECTOR_FOR_TITLE::text").get(),
                'price': product.css("YOUR_CSS_SELECTOR_FOR_PRICE::text").get(),
            }


    def closed(self, reason):
        self.driver.quit()